In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
import os
from datetime import datetime

# Import dataframes

In [ ]:
songs_df = pd.read_csv('songs.csv', sep='\t')
tracks_df = pd.read_csv('tracks.csv', sep='\t')
acoustic_features_df = pd.read_csv('acoustic_features.csv', sep='\t')
lyrics_df = pd.read_csv('lyrics.csv', sep='\t', quoting=3, error_bad_lines=False)
spotify_genre_mapping_df = pd.read_csv('spotify_genre_mapping.csv', sep='\t')
artists_df = pd.read_csv('artists.csv', sep='\t')
albums_df = pd.read_csv('albums.csv', sep='\t')
releases_df = pd.read_csv('releases.csv', sep='\t')
happiness_df = pd.read_csv('Happiness2017-2019.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


Create SongReleases table. 
Original data has artists as a dictionary converted to a string in the 'artists' column. 
Copy the song_id, song_name, billboard, song_type, num_artists from the original songs.csv. 
Create new columns artist_id, artist_name, num_artists.
  1. Convert the string of dictionary in 'artists' back to a dictionary
  2. For num_artists, count the number of keys in each dictionary
  3. Add new rows to the SongReleases table for each artist that performed the song. 
      The song is repeated in the dataframe for every artist that performed in it. 



In [ ]:
SongReleases_df = pd.DataFrame(columns = ['song_id', 'artist_id', 'song_type', 'num_artists'])
for i in songs_df.index:
  row_info = songs_df.loc[i]
  artist_dict = dict(literal_eval(songs_df.loc[i, 'artists']))
  num_artists = len(artist_dict.keys())
  for a_id, a_name in artist_dict.items(): 
    SongReleases_df.loc[len(SongReleases_df.index), :] = [row_info[0], a_id, row_info[6], num_artists]
SongReleases_df

,song_id,artist_id,song_type,num_artists
0,3e9HZxeyfWwjeyPAMmWSSQ,66CXWjxzNUsdJxJ2JdwvnR,Solo,1
1,5p7ujcrUXASCNwRaWNHR1C,26VFTg2z8YR0cCuwLzESi2,Solo,1
2,2xLMifQCjDGFmkHkpNLD9h,0Y5tJX1MQlPlqiwlOH1tJY,Solo,1
3,3KkXRkHbMCARz0aVfEt68P,246dkjvS1zLTtiykXe5h60,Collaboration,2
4,3KkXRkHbMCARz0aVfEt68P,1zNqQNIdeOUZHb8zbZRFMX,Collaboration,2
...,...,...,...,...
22234,4NnhLA66RRLXxKbiiscU9R,5X3TuTi9OIsJXMGxPwTKM2,Solo,1
22235,2jHfXdCLibrI1J56LnUAZv,6lHC2EQMEMZiEmSfFloarn,Solo,1
22236,6zqsyB7uIvWrL1iCJzpNrs,5X3TuTi9OIsJXMGxPwTKM2,Solo,1
22237,5mz9pQZZXNpAw9CdQ7Bk8q,6lHC2EQMEMZiEmSfFloarn,Solo,1


The isPartOf (renamed to AlbumTracks) dataframe.
Contains the album_id, song_id, track_number to associate songs with their albums.

In [ ]:
AlbumTracks_df = tracks_df.copy()
AlbumTracks_df = AlbumTracks_df.filter(items=['song_id', 'album_id', 'track_number'])
AlbumTracks_df.head()

,song_id,album_id,track_number
0,3e9HZxeyfWwjeyPAMmWSSQ,2fYhqwDWXjbpjaIJPEfKFw,11
1,5p7ujcrUXASCNwRaWNHR1C,0zzrCTzvL4ZmR42xF46Afm,1
2,2xLMifQCjDGFmkHkpNLD9h,41GuZcammIkupMPKH2OJ6I,3
3,3KkXRkHbMCARz0aVfEt68P,35s58BRTGAEWztPo9WqCIs,2
4,1rqqCSm0Qe4I9rUvWncaom,6ApYSpXF8GxZAgBTHDzYge,4


The Song dataframe.
Contains information on the song and its features. 
Information contained is [song_id, song_name, duration_ms, acousticness, danceability, energy, instrumentalness, liveness, loudness, speaciness, valence, tempo, explicit, lyrics]. 
Information extracted from: 
acoustic_features.csv: [song_id, duration_ms, acousticness, danceability, energy, instrumentalness, liveness, loudness, speaciness, valence, tempo]
songs.csv: [song_name, explicit] 
lyrics.csv: [lyrics]
Dataframes joined on matching song_id
Note: explicit was later changed to from boolean value to be 0 (not explicit) and 1 (explicit)

In [ ]:
Song_df = songs_df.merge(acoustic_features_df, how = 'inner', on = ['song_id'])
Song_df = Song_df.merge(lyrics_df, how = 'inner', on = ['song_id'])
Song_df = Song_df.filter(items=['song_id', 'song_name',
       'explicit', 'duration_ms', 'acousticness', 'danceability', 'energy', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'valence', 'tempo', 'lyrics'])


The Artist dataframe. 
'main_genre' that the artist is associated with is mapped to a simplified genre definition to simplify the types and allow for increased comparisons. 


In [ ]:
# Use the spotify_genre_mapping to create a dictionary
spotify_genre_mapping_dict_1 = spotify_genre_mapping_df.set_index('original_genre').T.to_dict()
spotify_genre_mapping_dict = {k: v['mapped_genre'] for k, v in spotify_genre_mapping_dict_1.items()}


In [ ]:
Artist_df = artists_df.copy()
Artist_df['genre'] = Artist_df['main_genre'].map(spotify_genre_mapping_dict)
Artist_df = Artist_df.filter(['artist_id', 'name', 'followers', 'popularity', 'artist_type', 'genre'])
Artist_df.loc[Artist_df['artist_type'] == '-', 'artist_type'] = np.nan

The Album dataframe. 
Contains the album_id, album_name, billboard name, total number of tracks, album_type (album or compilation). 

Album and AlbumReleases dataframe. 

Merge release_date from releases.csv with the Album information from albums.csv 

In [ ]:
simple_releases_df = releases_df.copy()
simple_releases_df = simple_releases_df.loc[simple_releases_df['release_date_precision'] == 'day'] #only keep albums with release dates that are days

albums_copy = albums_df.copy()
albums_copy = albums_copy.loc[albums_copy['album_type'] == 'album'] # only actual albums 

filt_alb_df = pd.DataFrame(columns = ['album_id', 'name', 'billboard', 'artist_id', 'total_tracks'])
for i in albums_copy.index:
  row_info = albums_copy.loc[i]
  artist_dict = dict(literal_eval(albums_copy.loc[i, 'artists']))
  num_artists = len(artist_dict.keys())
  if num_artists == 1: #only keep albums with one artist
    for a_id, a_name in artist_dict.items(): 
      filt_alb_df.loc[len(filt_alb_df.index), :] = [row_info[0], row_info[1], row_info[2], a_id, row_info[5]]

filt_alb_df = filt_alb_df.merge(simple_releases_df, how = 'inner', on = ['album_id', 'artist_id'])
Album_df = filt_alb_df.filter(['album_id', 'name', 'total_tracks'])
AlbumReleases_df = filt_alb_df.filter(['album_id', 'artist_id', 'release_date'])


In [ ]:
Album_df

,album_id,name,total_tracks
0,5n1GSzC1Reao29ScnpLYqp,Dying To Live,16
1,6UYZEYjpN1DYRW0kqFy9ZE,Championships,19
2,7uVimUILdzSZG4KKKWToq0,Christmas (Deluxe Special Edition),20
3,41GuZcammIkupMPKH2OJ6I,ASTROWORLD,17
4,6thZNGX8hUVSjUrqJgPB9b,Christmas Is Here!,12
...,...,...,...
18322,4ceeeNqKJ9OtF33DaHQuum,Songs Of The Beatles,13
18323,1G8AfOjrE0FO9w1gfemIy1,Shut Down (Remastered),12
18324,62usLEsQho4s5TCfa6Ks4s,Berlin '65/Paris '67,10
18325,4JkPhYJZ3EkoXUsdghYrHI,Great Themes from Hit Films,12


In [ ]:
AlbumReleases_df

,album_id,artist_id,release_date
0,5n1GSzC1Reao29ScnpLYqp,46SHBwWsqBkxI7EeeBEQG7,2018-12-14
1,6UYZEYjpN1DYRW0kqFy9ZE,20sxb77xiYeusSH8cVdatc,2018-11-30
2,7uVimUILdzSZG4KKKWToq0,1GxkXlMwML1oSg5eLPiAz3,2012-11-09
3,41GuZcammIkupMPKH2OJ6I,0Y5tJX1MQlPlqiwlOH1tJY,2018-08-03
4,6thZNGX8hUVSjUrqJgPB9b,26AHtbjWKiwYzsoGoUZq53,2018-11-29
...,...,...,...
18322,4ceeeNqKJ9OtF33DaHQuum,1bgyxtWjZwA5PQlDsvs9b8,1981-05-13
18323,1G8AfOjrE0FO9w1gfemIy1,3oDbviiivRWhXwIE8hxkVV,1964-03-02
18324,62usLEsQho4s5TCfa6Ks4s,4F7Q5NV6h5TSwCainz8S5A,1997-01-01
18325,4JkPhYJZ3EkoXUsdghYrHI,6OoJgs9H4YzzniwXUwY7Yc,2014-02-01


Putting all spotify top songs in a dataframe.

In [ ]:
country_name_map = {'us': 'United States', 'gl': 'Global', 'ca': 'Canada'}

In [ ]:
Spotify_Chart_df = pd.DataFrame(columns = ['country', 'start_date', 'end_date', 'position', 'song_id', 'streams'])

In [ ]:
dirs = ['spotify_us_2018', 'spotify_us_2017']
for dir in dirs: 
  dir_list = os.listdir('./' + dir)
  for fname in dir_list: 
    country = country_name_map[fname[0:2]]
    start_date = fname[24:34]
    end_date = fname[36:46]
    this_df = pd.read_csv('./' + dir + '/' + fname, sep='\t')

    for i in this_df.index:
      row_info = this_df.loc[i]
      Spotify_Chart_df.loc[len(Spotify_Chart_df.index), :] = [country, start_date, end_date, row_info[0], row_info[1], row_info[4]]
Spotify_Chart_df

,country,start_date,end_date,position,song_id,streams
0,United States,2018-08-03,2018-08-10,1,2xLMifQCjDGFmkHkpNLD9h,19657392
1,United States,2018-08-03,2018-08-10,2,2G7V7zsVDxg1yRsu7Ew9RJ,16740738
2,United States,2018-08-03,2018-08-10,3,7wBJfHzpfI3032CSD7CE2m,16135390
3,United States,2018-08-03,2018-08-10,4,2E124GmJRnBJuXbTb4cPUB,13705250
4,United States,2018-08-03,2018-08-10,5,7KZ5MMVgBVox9ycroB2UrI,10496055
...,...,...,...,...,...,...
20795,United States,2017-12-08,2017-12-15,196,2bjwRfXMk4uRgOD9IBYl9h,1404284
20796,United States,2017-12-08,2017-12-15,197,6mapJIPnQ23RTAevUoE0DL,1399555
20797,United States,2017-12-08,2017-12-15,198,7faDzZnZYqTyYThx2sbHVQ,1396240
20798,United States,2017-12-08,2017-12-15,199,3NdDpSvN911VPGivFlV5d0,1394839


Country Happiness data was merged using excel. I selected data from the relevant countries and years to insert into the dataset. 

Create smaller versions of the datasets. 

In [ ]:
small_spotify_chart = Spotify_Chart_df[20:30]
small_spotify_chart

In [ ]:
song_ids_on_chart = small_spotify_chart['song_id']
song_ids_on_chart
small_song_df_1 = Song_df[20:28]
small_song_df_2 = Song_df.merge(song_ids_on_chart, how = 'inner', on = ['song_id'])
small_song_df = small_song_df_1.append(small_song_df_2, ignore_index=True)
small_song_df = small_song_df.drop_duplicates(keep='last')
small_song_df


In [ ]:
albums_of_songs_on_chart = AlbumTracks_df.merge(song_ids_on_chart, how = 'inner', on = ['song_id'])
albums_of_songs_on_chart = albums_of_songs_on_chart['album_id']
albums_of_songs_on_chart = albums_of_songs_on_chart[0:4]
small_albumtracks_df_1 = AlbumTracks_df[30:33]
small_albumtracks_df_2 = AlbumTracks_df.merge(albums_of_songs_on_chart, how = 'inner', on = ['album_id'])
small_albumtracks_df  = small_albumtracks_df_1.append(small_albumtracks_df_2, ignore_index=True)
small_albumtracks_df = small_albumtracks_df.drop_duplicates(keep='last')
small_albumtracks_df = small_albumtracks_df.sample(frac=.75).reset_index(drop=True)
small_albumtracks_df

In [ ]:
artists_with_songs_on_chart = SongReleases_df.merge(song_ids_on_chart, how = 'inner', on = ['song_id'])
artists_with_songs_on_chart = artists_with_songs_on_chart['artist_id']
small_artists_df_1 = Artist_df.merge(artists_with_songs_on_chart, how = 'inner', on = ['artist_id'])
artists_with_albums_on_chart = AlbumReleases_df.merge(albums_of_songs_on_chart, how = 'inner', on = ['album_id'])
artists_with_albums_on_chart = artists_with_albums_on_chart['artist_id']
small_artists_df_2 = Artist_df.merge(artists_with_albums_on_chart, how = 'inner', on = ['artist_id'])
small_artists_df_3 = Artist_df[50:58]
small_artists_df  = small_artists_df_1.append(small_artists_df_2, ignore_index=True)
small_artists_df  = small_artists_df.append(small_artists_df_3, ignore_index=True)
small_artists_df = small_artists_df.drop_duplicates(keep='last')
small_artists_df

In [ ]:
albums_of_songs_on_chart = AlbumTracks_df.merge(song_ids_on_chart, how = 'inner', on = ['song_id'])
albums_of_songs_on_chart = albums_of_songs_on_chart['album_id']
small_album_releases_1 = AlbumReleases_df.merge(albums_of_songs_on_chart, how = 'inner', on = ['album_id'])
small_album_releases_2 = AlbumReleases_df.merge(small_artists_df['artist_id'][0:3], how = 'inner', on = ['artist_id'])
small_album_releases_3 = AlbumReleases_df[50:57]
small_album_releases_df  = small_album_releases_1.append(small_album_releases_2, ignore_index=True)
small_album_releases_df  = small_album_releases_df.append(small_album_releases_3, ignore_index=True)
small_album_releases_df = small_album_releases_df.drop_duplicates(keep='last')
small_album_releases_df

In [ ]:
albums_of_songs_on_chart = AlbumTracks_df.merge(song_ids_on_chart, how = 'inner', on = ['song_id'])
albums_of_songs_on_chart = albums_of_songs_on_chart['album_id']
small_album_df = Album_df.merge(albums_of_songs_on_chart, how = 'inner', on = ['album_id'])
small_album_df = small_album_df.append(Album_df[10:15], ignore_index=True)
small_album_df = small_album_df.append(Album_df[50:55], ignore_index=True)
small_album_df = small_album_df.drop_duplicates(keep='last')
small_album_df

In [ ]:
song_releases_df_1 = SongReleases_df.merge(song_ids_on_chart, how = 'inner', on = ['song_id'])
song_releases_df_2 = SongReleases_df.merge(small_artists_df[5:10], how = 'inner', on = ['artist_id'])
small_song_releases_df = song_releases_df_1.append(song_releases_df_2, ignore_index=True)
small_song_releases_df = small_song_releases_df.drop_duplicates(keep='last')
small_song_releases_df = small_song_releases_df.sample(frac=.5).reset_index(drop=True)
small_song_releases_df

In [ ]:
small_spotify_chart.to_csv('SpotifyChart-small.txt', index=None, header=None, sep='\t', mode='a')
small_song_df.to_csv('Song-small.txt', index=None, header=None, sep='\t', mode='a')
small_albumtracks_df.to_csv('AlbumTracks-small.txt', index=None, header=None, sep='\t', mode='a')
small_artists_df.to_csv('Artist-small.txt', index=None, header=None, sep='\t', mode='a')
small_album_releases_df.to_csv('AlbumReleases-small.txt', index=None, header=None, sep='\t', mode='a')
small_album_df.to_csv('Album-small.txt', index=None, header=None, sep='\t', mode='a')
small_song_releases_df.to_csv('SongReleases-small.txt', index=None, header=None, sep='\t', mode='a')
happiness_df.to_csv('CountryHappiness-small.txt', index=None, header=None, sep='\t', mode='a')

In [ ]:
!zip -r /content/SmallDataFrames.zip /content/SmallDataFrames

  adding: content/SmallDataFrames/ (stored 0%)
  adding: content/SmallDataFrames/SongReleases-small.txt (deflated 72%)
  adding: content/SmallDataFrames/SpotifyChart-small.txt (deflated 55%)
  adding: content/SmallDataFrames/Song-small.txt (deflated 70%)
  adding: content/SmallDataFrames/Album-small.txt (deflated 18%)
  adding: content/SmallDataFrames/Artist-small.txt (deflated 28%)
  adding: content/SmallDataFrames/CountryHappiness-small.txt (deflated 77%)
  adding: content/SmallDataFrames/AlbumTracks-small.txt (deflated 53%)
  adding: content/SmallDataFrames/AlbumReleases-small.txt (deflated 41%)


In [ ]:
Spotify_Chart_df.to_csv('SpotifyChart.txt', index=None, header=None, sep='\t', mode='a')
Song_df.to_csv('Song.txt', index=None, header=None, sep='\t', mode='a')
AlbumTracks_df.to_csv('AlbumTracks.txt', index=None, header=None, sep='\t', mode='a')
Artist_df.to_csv('Artist.txt', index=None, header=None, sep='\t', mode='a')
AlbumReleases_df.to_csv('AlbumReleases.txt', index=None, header=None, sep='\t', mode='a')
Album_df.to_csv('Albuml.txt', index=None, header=None, sep='\t', mode='a')
SongReleases_df.to_csv('SongReleases.txt', index=None, header=None, sep='\t', mode='a')
happiness_df.to_csv('CountryHappiness.txt', index=None, header=None, sep='\t', mode='a')
!zip -r /content/DataFrames.zip /content/*.txt


  adding: content/Albuml.txt (deflated 37%)
  adding: content/AlbumReleases.txt (deflated 40%)
  adding: content/AlbumTracks.txt (deflated 35%)
  adding: content/Artist.txt (deflated 39%)
  adding: content/CountryHappiness.txt (deflated 55%)
  adding: content/SongReleases.txt (deflated 54%)
  adding: content/Song.txt (deflated 64%)
  adding: content/SpotifyChart.txt (deflated 75%)
